In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc    
from dash import html
import plotly.express as px
import base64
from dash import dash_table
from dash.dependencies import Input, Output, State

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter class from your CRUD module
from animal_shelter_CRUD_original import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Use your actual MongoDB username and password
username = "edwardgarcia5_snhu"
password = "password"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password, 'host.docker.internal', 27017, 'AAC', 'animals')

# Retrieve all records from the database
df = pd.DataFrame.from_records(shelter.read({}))

# Drop the '_id' column only if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load the Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your image filename
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the layout of the dashboard
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.A(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'15%', 'width':'15%'}),
        
    )),
    html.Center(html.H2('Developed by Edward Garcia')),
    html.Hr(),

    # Interactive filter options using RadioItems to resemble buttons
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'},
            style={'font-size': '20px', 'text-align': 'center'}
        )
    ], style={'padding': '10px', 'background-color': '#f9f9f9', 'border-radius': '5px'}),

    html.Hr(),

    # Enhanced DataTable with Pagination, Sorting, and Filtering
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single",
        selected_rows=[0],
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10,
        style_data_conditional=[
            {
                'if': {'column_id': i},
                'background_color': '#D2F3FF'
            } for i in df.columns
        ],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
    ),
    html.Br(),
    html.Hr(),

    # Div for the charts and map side by side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update DataTable based on filter selection
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        dff = shelter.read({"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}})
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = shelter.read({"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Border Collie", "Siberian Husky",]}})
    elif filter_type == 'Disaster or Individual Tracking':
        dff = shelter.read({"breed": {"$in": ["Doberman Pinscher", "Bloodhound", "Rottweiler"]}})
    else:
        dff = shelter.read({})
    
    df_filtered = pd.DataFrame.from_records(dff)
    
    # Drop the '_id' column only if it exists in the filtered data
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    
    return df_filtered.to_dict('records')

# Callback to update the chart based on the filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData:
        dff = pd.DataFrame.from_dict(viewData)
        return [
            dcc.Graph(            
                figure = px.pie(dff, names='breed', title='Breed Distribution')
            )    
        ]
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Breed Distribution')
        )    
    ]

# Callback to update the map based on the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(selected_rows):
    if not selected_rows:  # Build a default view if there are no selected lines
        markerArray = (30.75, -97.48)  # Default marker at Austin Animal Shelter
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
    else:  # Build the contextual views based on the selection
        dff = pd.DataFrame(df.iloc[selected_rows])  # Convert the datatable to a dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1])  # Strip out the lat
        coordLong = float(dff['location_long'].to_string().split()[1])  # Strip out the long
        markerArray = (coordLat, coordLong)  # Build the array based on selection
        
        toolTip = dff['breed'].values[0]  # Ensure correct type for tooltip
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name'].values[0]  # Ensure correct type for popup paragraph

    return [dl.Map(style={'width': '600px', 'height': '300px'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]

# Run the Dash app
app.run_server(debug=True)


C:\Python312\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
